In [73]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
from datetime import timedelta
from sklearn.impute import SimpleImputer
import numpy as np

In [74]:
transportData = "Refrigerated_Truck_Rates_and_Availability_Full.csv"
AvoTransportData = "avocado_tranpsort.csv"
avo_data = "avo_prices.csv"
gas_data = "gas_prices.csv"
loadweather = "NewSDweatherData6.csv"
bananaprices = "banana_prices.csv"

In [75]:
banana_prices_df = pd.read_csv(bananaprices)
banana_prices_df.head()

,Date,Banana Price per Pound,old,Percent Change
0,1/1/2016,0.581,-0.003,-0.3
1,2/1/2016,0.573,-0.030,-3.0
2,3/1/2016,0.586,-0.012,-1.2
3,4/1/2016,0.574,-0.039,-3.9
4,5/1/2016,0.570,-0.021,-2.1


In [76]:
SDweather_df = pd.read_csv(loadweather)
SDweather_df.head()

,dt_iso,dt,timezone,lat,lon,temp,feels_like,temp_min,temp_max,pressure,...,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id
0,2015-01-01,1.420112e+09,-28800.0,32.715738,-117.161084,46.119167,38.966667,42.921667,51.287917,1019.291667,...,NaN,45.958333,5.002083,160.583333,NaN,NaN,NaN,NaN,14.541667,800.708333
1,2015-01-02,1.420198e+09,-28800.0,32.715738,-117.161084,48.741250,42.147917,45.371250,53.362917,1020.291667,...,NaN,54.500000,5.635833,144.583333,NaN,NaN,NaN,NaN,4.166667,800.166667
2,2015-01-03,1.420285e+09,-28800.0,32.715738,-117.161084,50.742917,45.510000,47.449167,54.095000,1021.916667,...,NaN,62.583333,4.790833,204.250000,NaN,NaN,NaN,NaN,21.041667,801.041667
3,2015-01-04,1.420371e+09,-28800.0,32.715738,-117.161084,52.755417,47.413333,50.084583,56.322917,1025.166667,...,NaN,63.708333,5.609167,185.416667,NaN,NaN,NaN,NaN,18.458333,800.875000
4,2015-01-05,1.420457e+09,-28800.0,32.715738,-117.161084,58.379167,53.038333,55.012917,62.826667,1023.375000,...,NaN,59.041667,6.302500,199.208333,NaN,NaN,NaN,NaN,29.708333,775.458333


In [77]:
initial_transport_df = pd.read_csv(transportData)
initial_transport_df.head()

,Date,Week,Month,Quarter,Year,Region,Origin,Destination,Distance,Commodity,Week Low,Week High,Midpoint,Rate Per Mile,Availability
0,01/07/2020,1,1,1,2020,Arizona,"Imperial, Palo Verde And Coachella Valleys, Ca...",ATLANTA,2100.0,"LETTUCE, BROCCOLI, CAULIFLOWER, ROMAINE AND LE...",5000,5400,5200,2.48,5
1,01/07/2020,1,1,1,2020,Arizona,"Imperial, Palo Verde And Coachella Valleys, Ca...",BALTIMORE,2600.0,"LETTUCE, BROCCOLI, CAULIFLOWER, ROMAINE AND LE...",6400,6800,6600,2.54,5
2,01/07/2020,1,1,1,2020,Arizona,"Imperial, Palo Verde And Coachella Valleys, Ca...",BOSTON,2900.0,"LETTUCE, BROCCOLI, CAULIFLOWER, ROMAINE AND LE...",7300,7800,7550,2.60,5
3,01/07/2020,1,1,1,2020,Arizona,"Imperial, Palo Verde And Coachella Valleys, Ca...",CHICAGO,2030.0,"LETTUCE, BROCCOLI, CAULIFLOWER, ROMAINE AND LE...",4700,5200,4950,2.44,5
4,01/07/2020,1,1,1,2020,Arizona,"Imperial, Palo Verde And Coachella Valleys, Ca...",DALLAS,1300.0,"LETTUCE, BROCCOLI, CAULIFLOWER, ROMAINE AND LE...",3600,4100,3850,2.96,5


In [78]:
initial_avo_transport_df = pd.read_csv(AvoTransportData)
# initial_avo_transport_df.head()
initial_avo_transport_df.head()

,Date,Week,Month,Quarter,Year,Region,Origin,Destination,Distance,Commodity,Week Low,Week High,Midpoint,Rate Per Mile,Availability
0,01/07/2020,1,1,1,2020,California,South District California,ATLANTA,2100,"CITRUS, AVOCADOS",4500,5500,5100,2.43,3
1,01/07/2020,1,1,1,2020,California,South District California,BALTIMORE,2600,"CITRUS, AVOCADOS",5800,6800,6300,2.42,3
2,01/07/2020,1,1,1,2020,California,South District California,BOSTON,2900,"CITRUS, AVOCADOS",6600,7500,6900,2.38,3
3,01/07/2020,1,1,1,2020,California,South District California,CHICAGO,2030,"CITRUS, AVOCADOS",4100,5000,4500,2.22,3
4,01/07/2020,1,1,1,2020,California,South District California,DALLAS,1300,"CITRUS, AVOCADOS",3200,4200,3400,2.62,3


In [79]:
avo_data_df = pd.read_csv(avo_data)


merge_avo_df = avo_data_df.loc[avo_data_df['region'] == 'TotalUS']
merge_avo_df = merge_avo_df.loc[merge_avo_df['type'] == 'conventional']
merge_avo_df = merge_avo_df[['New Date', 'AveragePrice']]

In [80]:
gas_data_df = pd.read_csv(gas_data)
gas_data_df.head()

,Date,Gasoline - All Grades,All Grades - Conventional Areas,All Grades - Reformulated Areas,Regular,Conventional Areas - Regular,Reformulated Areas - Regular,Midgrade,Conventional Areas - Midgrade,Reformulated Areas - Midgrade,Premium,Conventional Areas - Premium,Reformulated Areas - Premium,Diesel (On-Highway) - All Types,Ultra Low Sulfur (15 ppm and Under),Low Sulfer (Above 15 ppm to 500 ppm)
0,8/20/1990,NaN,NaN,NaN,1.191,1.191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8/27/1990,NaN,NaN,NaN,1.245,1.245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9/3/1990,NaN,NaN,NaN,1.242,1.242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9/10/1990,NaN,NaN,NaN,1.252,1.252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9/17/1990,NaN,NaN,NaN,1.266,1.266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
merge_avo_df.head()

,New Date,AveragePrice
51,1/5/2015,0.95
159,1/12/2015,1.01
267,1/19/2015,1.03
375,1/26/2015,1.04
483,2/2/2015,0.89


In [82]:
merge_avo_df = merge_avo_df.rename(columns={"New Date": "Date",
                                                        "AveragePrice": "Avo Average Price",
                                                   })

In [83]:
merge_avo_df = merge_avo_df.reset_index()

In [84]:
merge_avo_df = merge_avo_df[['Date', 'Avo Average Price']]
merge_avo_df.head()

,Date,Avo Average Price
0,1/5/2015,0.95
1,1/12/2015,1.01
2,1/19/2015,1.03
3,1/26/2015,1.04
4,2/2/2015,0.89


In [85]:
gas_data_df['Date'].dtype

dtype('O')

In [86]:
gas_data_df['Date'] = pd.to_datetime(gas_data_df['Date'])
merge_avo_df['Date'] = pd.to_datetime(merge_avo_df['Date'])

In [87]:
gas_data_df = gas_data_df[['Date', 'Gasoline - All Grades']]
gas_data_df

,Date,Gasoline - All Grades
0,1990-08-20,NaN
1,1990-08-27,NaN
2,1990-09-03,NaN
3,1990-09-10,NaN
4,1990-09-17,NaN
...,...,...
1556,2020-06-15,2.185
1557,2020-06-22,2.216
1558,2020-06-29,2.260
1559,2020-07-06,2.265


In [88]:
initial_avo_transport_df['Date'] = pd.to_datetime(initial_avo_transport_df['Date'])
initial_avo_transport_df.head()

,Date,Week,Month,Quarter,Year,Region,Origin,Destination,Distance,Commodity,Week Low,Week High,Midpoint,Rate Per Mile,Availability
0,2020-01-07,1,1,1,2020,California,South District California,ATLANTA,2100,"CITRUS, AVOCADOS",4500,5500,5100,2.43,3
1,2020-01-07,1,1,1,2020,California,South District California,BALTIMORE,2600,"CITRUS, AVOCADOS",5800,6800,6300,2.42,3
2,2020-01-07,1,1,1,2020,California,South District California,BOSTON,2900,"CITRUS, AVOCADOS",6600,7500,6900,2.38,3
3,2020-01-07,1,1,1,2020,California,South District California,CHICAGO,2030,"CITRUS, AVOCADOS",4100,5000,4500,2.22,3
4,2020-01-07,1,1,1,2020,California,South District California,DALLAS,1300,"CITRUS, AVOCADOS",3200,4200,3400,2.62,3


In [89]:
initial_avo_transport_df = initial_avo_transport_df[['Date', 'Rate Per Mile']]
initial_avo_transport_df.head()

,Date,Rate Per Mile
0,2020-01-07,2.43
1,2020-01-07,2.42
2,2020-01-07,2.38
3,2020-01-07,2.22
4,2020-01-07,2.62


In [90]:
initial_avo_transport_df = initial_avo_transport_df.groupby(by="Date").mean()

initial_avo_transport_df.head()

,Rate Per Mile
Date,
2003-11-04,1.228
2003-11-12,1.272
2003-11-18,1.300
2003-11-25,1.318
2003-12-02,1.284


In [91]:
initial_avo_transport_df = initial_avo_transport_df.reset_index()
initial_avo_transport_df.head()

,Date,Rate Per Mile
0,2003-11-04,1.228
1,2003-11-12,1.272
2,2003-11-18,1.300
3,2003-11-25,1.318
4,2003-12-02,1.284


In [92]:
initial_avo_transport_df['Date'] = initial_avo_transport_df['Date'] - timedelta(days=1)
initial_avo_transport_df.head()

,Date,Rate Per Mile
0,2003-11-03,1.228
1,2003-11-11,1.272
2,2003-11-17,1.300
3,2003-11-24,1.318
4,2003-12-01,1.284


In [93]:
final_df = pd.merge(initial_avo_transport_df, gas_data_df, how='outer', on=['Date'])
final_df.head()

,Date,Rate Per Mile,Gasoline - All Grades
0,2003-11-03,1.228,1.577
1,2003-11-11,1.272,NaN
2,2003-11-17,1.300,1.540
3,2003-11-24,1.318,1.554
4,2003-12-01,1.284,1.533


In [94]:
final_df = pd.merge(final_df, merge_avo_df, how='outer', on=['Date'])
final_df.tail()

,Date,Rate Per Mile,Gasoline - All Grades,Avo Average Price
1573,2017-02-13,NaN,2.418,0.87
1574,2017-07-03,NaN,2.376,1.21
1575,2019-03-18,NaN,2.625,NaN
1576,2019-03-25,NaN,2.701,NaN
1577,2019-12-23,NaN,2.621,NaN


In [95]:
SDweather_df.head()

,dt_iso,dt,timezone,lat,lon,temp,feels_like,temp_min,temp_max,pressure,...,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id
0,2015-01-01,1.420112e+09,-28800.0,32.715738,-117.161084,46.119167,38.966667,42.921667,51.287917,1019.291667,...,NaN,45.958333,5.002083,160.583333,NaN,NaN,NaN,NaN,14.541667,800.708333
1,2015-01-02,1.420198e+09,-28800.0,32.715738,-117.161084,48.741250,42.147917,45.371250,53.362917,1020.291667,...,NaN,54.500000,5.635833,144.583333,NaN,NaN,NaN,NaN,4.166667,800.166667
2,2015-01-03,1.420285e+09,-28800.0,32.715738,-117.161084,50.742917,45.510000,47.449167,54.095000,1021.916667,...,NaN,62.583333,4.790833,204.250000,NaN,NaN,NaN,NaN,21.041667,801.041667
3,2015-01-04,1.420371e+09,-28800.0,32.715738,-117.161084,52.755417,47.413333,50.084583,56.322917,1025.166667,...,NaN,63.708333,5.609167,185.416667,NaN,NaN,NaN,NaN,18.458333,800.875000
4,2015-01-05,1.420457e+09,-28800.0,32.715738,-117.161084,58.379167,53.038333,55.012917,62.826667,1023.375000,...,NaN,59.041667,6.302500,199.208333,NaN,NaN,NaN,NaN,29.708333,775.458333


In [96]:
weather_df = SDweather_df[['dt_iso', 'temp']]
weather_df = weather_df.rename(columns={"dt_iso": "Date",
                                                        "temp": "Temp (F)",
                                                   })

In [97]:
weather_df

,Date,Temp (F)
0,2015-01-01,46.119167
1,2015-01-02,48.741250
2,2015-01-03,50.742917
3,2015-01-04,52.755417
4,2015-01-05,58.379167
...,...,...
1426,2018-11-27,63.020417
1427,2018-11-28,61.155000
1428,2018-11-29,61.515000
1429,2018-11-30,61.797500


In [99]:
weather_df['Date'] = pd.to_datetime(weather_df['Date'])

In [100]:
final_df = pd.merge(final_df, weather_df, how='inner', on=['Date'])
final_df.head()

,Date,Rate Per Mile,Gasoline - All Grades,Avo Average Price,Temp (F)
0,2015-05-11,2.663333,2.776,1.02,63.347500
1,2015-05-18,2.672222,2.827,1.06,61.847917
2,2015-05-25,2.661111,2.857,1.04,62.741667
3,2015-06-01,2.731111,2.863,1.03,64.777083
4,2015-06-08,2.741111,2.863,1.00,67.384167


In [106]:
date_df = pd.read_csv('date.csv')
date_df['Date'] = pd.to_datetime(date_df['Date'])
date_df

,Date
0,2015-01-05
1,2015-01-12
2,2015-01-19
3,2015-01-26
4,2015-02-02
...,...
199,2018-10-29
200,2018-11-05
201,2018-11-12
202,2018-11-19


In [107]:
final_df = pd.merge(date_df, final_df, how='outer', on=['Date'])
final_df.head()

,Date,Rate Per Mile,Gasoline - All Grades,Avo Average Price,Temp (F)
0,2015-01-05,NaN,2.308,0.95,58.379167
1,2015-01-12,NaN,2.232,1.01,60.339630
2,2015-01-19,NaN,2.157,1.03,58.188333
3,2015-01-26,NaN,2.133,1.04,64.723750
4,2015-02-02,NaN,2.154,0.89,60.628333


In [109]:
final_df.to_csv("final_df.csv", index=False)